<a href="https://colab.research.google.com/github/Electromensch/nlp_need/blob/main/text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.7.14


In [1]:
import nltk

In [2]:
import pickle

In [3]:
from tqdm import tqdm
from collections import defaultdict
from nltk.corpus import stopwords

In [4]:
import numpy as np
import re
import pandas as pd
from collections import Counter
from copy import deepcopy

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_pickle('/content/drive/MyDrive/mypickle3')

In [ ]:
df.head()

,sentences,text,ner,tokens,tags,ner_new,ner_final
0,"Вот принесли эти задания в колледж, надо решит...",Tesak,"[(Вот, O), (принесли, O), (эти, O), (задания, ...","[Вот, принесли, эти, задания, в, колледж, ,, н...","[(Вот, PRCL), (принесли, VERB), (эти, ADJF), (...","[(Вот, O), (принесли, O), (эти, O), (задания, ...",[]
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak,"[(Когда-то, O), (у, O), (меня, O), (по, O), (х...","[Когда-то, у, меня, по, химии, была, пятерка, ...","[(Когда-то, ADVB), (у, PREP), (меня, NPRO), (п...","[(Когда-то, O), (у, O), (меня, O), (по, O), (х...",[]
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak,"[(На, O), (зоне, O), (не, O), (нужно, O), (бол...","[На, зоне, не, нужно, больше, знать, и, уметь,...","[(На, PREP), (зоне, NOUN), (не, PRCL), (нужно,...","[(На, O), (зоне, O), (не, O), (нужно, O), (бол...",[]
3,"Только я сказал, сразу меня начали просить по...",Tesak,"[(Только, O), (я, O), (сказал, O), (,, O), (ср...","[Только, я, сказал, ,, сразу, меня, начали, пр...","[(Только, PRCL), (я, NPRO), (сказал, VERB), (,...","[(Только, O), (я, O), (сказал, O), (,, O), (ср...",[]
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak,"[(Мы, O), (садимся, O), (,, O), (я, O), (откры...","[Мы, садимся, ,, я, открываю, эти, задачи, и, ...","[(Мы, NPRO), (садимся, VERB), (,, None), (я, N...","[(Мы, O), (садимся, O), (,, O), (я, O), (откры...",[]


In [7]:
my_gram = 4

In [8]:
def pad(x, char, n=1, left=True):
    if left:
        return ' '.join(char for i in range(n)) +" " + x
    else:
        return x+" "+ ' '.join(char for i in range(n)) 
    
def process_data(x, start_token='<start>', end_token='<end>', lower=False, n_gram=2):
    n_gram = max(n_gram,2)
    if isinstance(x, str):
        x = re.split(r'[.?!]\s*',x)
    x = [re.sub('[^\w]+', ' ', sentence).strip() for sentence in x]
    if lower:
        x = [sentence.lower() for sentence in x]
    x = [sentence.replace('.','').replace(',','').replace('!','').replace('?','').strip() for sentence in x if sentence]
    x = [pad(sentence, start_token, n_gram-1, left=True) for sentence in x if sentence]
    x = [pad(sentence, end_token, 1, left=False) for sentence in x if sentence]
    return x

In [9]:
from tqdm import tqdm
from collections import defaultdict

In [10]:
df1 = df[df.text == 'Tesak']

In [11]:
df2 = df[df.text == 'Koran'].sample(len(df1), random_state=0)

In [12]:
df_bal = pd.concat([df1, df2])

In [13]:
processed_texts = process_data(df_bal.sentences, n_gram=my_gram)

In [14]:
class MarkovChains:
    def __init__(self, n_grams=2, texts=None):
        self.n_grams = n_grams
        self.counter_dict = None
        self.n_tokens = None
        self.vocab = None
        self.inverse_vocab = None
        self.texts = texts
        if not self.texts is None:
            self.counter_dict, self.vocab, self.n_tokens = self.build_vocab()
        self.dict_probs = defaultdict(dict)
        
    def build_vocab(self):
        if isinstance(self.texts, list):
            vocab = list(set(np.hstack([i.split(' ') for i in self.texts])))
        elif isinstance(str): 
            vocab = list(set(self.texts.split(' ')))
            self.texts = [pad(i.strip(), '<end>', left=False) for i in self.texts.split('<end>')]
        counter_dict = Counter([])
        for sentence in tqdm(processed_texts):
            tokens = sentence.split(' ')
            for l in range(len(tokens)-self.n_grams+2):
                counter_dict.update([' '.join(tokens[l:l+self.n_grams-1])])
        n_tokens = len(vocab)
        print('Build vocabulary')
        return counter_dict, vocab, n_tokens
        
    def train(self, texts=None):
        if texts is None:
            assert self.texts, 'Please provide texts data'
        else:
            self.texts = texts
        if self.vocab is None:
            self.counter_dict, self.vocab, self.n_tokens = self.build_vocab()
        n_gram_counter = Counter([])
        for sentence in tqdm(processed_texts):
            tokens = sentence.split(' ')
            for l in range(len(tokens)-self.n_grams+1):
                n_gram_counter.update([' '.join(tokens[l:l+self.n_grams])])
        for n_gram, freq in tqdm(n_gram_counter.items()):
            tokens = n_gram.split(' ')
            main_token = tokens[-1]
            cond_tokens = ' '.join(tokens[:-1])
            cond_tokens_freq = self.counter_dict[cond_tokens]
            self.dict_probs[cond_tokens][main_token] = freq/cond_tokens_freq
        print('Finished training')
    
    def generate(self, start_token='<start>', end_token='<end>', greedy=False,
                 decoding='random', beam_size=2, max_sent_size=15):
      if start_token == '<start>':
        start_token2 = [start_token for i in range(self.n_grams-1)]
        generated_tokens = [' '.join(start_token2)]
        n_seq = 1
      else:
        start_token2 = ['<start>' for i in range(self.n_grams-2)]
        start_token2.append(start_token)
        generated_tokens = [' '.join(start_token2)]
        n_seq = 2
      if decoding == 'random':
        next_token = start_token
        while True:
            generated_tokens = list(np.hstack(i.split(' ') for i in generated_tokens))
            probs = self.dict_probs[' '.join(generated_tokens[-self.n_grams+1:])] 
            variants = list(probs.keys())
            probs = list(probs.values())
            if greedy:
              idx = np.argmax(probs)
            else:
              idx = np.random.choice(range(len(variants)), p=probs)
            next_token = variants[idx]
            if next_token==end_token:
              break
            generated_tokens.append(next_token)
        generated_tokens.append('.')
        generation = ' '.join(generated_tokens[self.n_grams-n_seq:]).capitalize()
        return generation
        
      elif decoding == 'beam_search':
        probs = [[0]]
        sentences = [start_token2]
        while True:
          new_probs = []
          new_sentences = []
          for c, sentence in enumerate(sentences):
            if len(sentence) <= max_sent_size or sentence[-1] != '<end>':
              start_token2 = sentence[-self.n_grams+1:]
              all_probs = self.dict_probs[' '.join(start_token2)] 
              variants = list(all_probs.keys())
              all_probs = np.array(list(all_probs.values()))
              top_words = [variants[i] for i in all_probs.argsort()[-beam_size:]]
              top_probs = [all_probs[i] for i in all_probs.argsort()[-beam_size:]]

              new_sentences.extend([sentence + [i] for i in top_words])
              new_probs.extend([probs[c]+[i] for i in top_probs])
            else:
              new_sentences.append(sentence)
              new_probs.append(probs[c])
            
          sentences = deepcopy(new_sentences)
          probs = deepcopy(new_probs)

          rules = [len(sentence) >= max_sent_size or sentence[-1]=='<end>' for sentence in sentences]
            
          if all(rules):
            break

        idx = np.argmax(np.sum(probs, axis=1))
        sentence = ' '.join(sentences[idx]).replace('<start> ', '').replace('<end>' , '')
        sentence += '.'
        return sentence
    
    def get_params(self):
        return self.dict_probs

In [15]:
model4 = MarkovChains(n_grams=my_gram, texts=processed_texts)

100%|██████████| 34956/34956 [00:01<00:00, 27075.97it/s]

Build vocabulary


In [16]:
model4.train()

100%|██████████| 416358/416358 [00:01<00:00, 388942.94it/s]

Finished training


In [18]:
with open('model4pkl', 'wb') as f:
    pickle.dump(model4, f)

In [19]:
!pip freeze > requirements.txt

In [17]:
prikol = model.generate(start_token='Вот', decoding='beam_search', beam_size=5)
prikol

'Вот так в каждом селении Мы создавали главных грешников чтобы они строили там.'

In [ ]:
with open('/content/drive/MyDrive/list of prikols.txt', 'a', encoding = 'utf-8') as f:
  f.write(prikol + '\n')